In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy
import statsmodels.api as sm 
import datetime
import os
import yfinance as yf # https://aroussi.com/post/python-yahoo-finance

print('Analytics Packages Imported')

Analytics Packages Imported


In [2]:
# Fetch historical data for multiple tickers
data = yf.download("META TWLO", start='2021-01-01', end='2024-03-22', group_by="ticker")
data.head()

[*********************100%%**********************]  2 of 2 completed


Ticker            META                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2021-01-04  274.779999  275.000000  265.200012  268.940002  268.654968   
2021-01-05  268.290009  272.399994  268.209991  270.970001  270.682800   
2021-01-06  262.000000  267.750000  260.010010  263.309998  263.030914   
2021-01-07  265.899994  271.609985  264.779999  268.739990  268.455170   
2021-01-08  268.309998  268.950012  263.179993  267.570007  267.286407   

Ticker                      TWLO                                      \
Price         Volume        Open        High         Low       Close   
Date                                                                   
2021-01-04  15106100  339.000000  340.596008  326.260010  334.589996   
2021-01-05   9871600  330.429993  341.790009  329.000000  341.239990   
2021-01-06  24354100  329.089996  337.929993  326.109985  329.010010   
2021-01-07  15789800  339.880005  357.500000  337.315002  357.149994   
2021-01-08  18528300  362.339996  373.005005  355.790009  360.269989   

Ticker                           
Price        Adj Close   Volume  
Date                             
2021-01-04  334.589996  1968900  
2021-01-05  341.239990  1375400  
2021-01-06  329.010010  1837600  
2021-01-07  357.149994  2057900  
2021-01-08  360.269989  2136200

In [3]:
# The data is a bit funky as it is wide (each ticker's data in a separate column)
# Let's convert it to a long format (where the ticker symbol is included in each row alongside the data)

data_stacked = data.stack(level=0).reset_index().rename(columns={"level_1": "Ticker"})

print(data_stacked.head())

Price       Date Ticker   Adj Close       Close        High         Low  \
0     2021-01-04   META  268.654968  268.940002  275.000000  265.200012   
1     2021-01-04   TWLO  334.589996  334.589996  340.596008  326.260010   
2     2021-01-05   META  270.682800  270.970001  272.399994  268.209991   
3     2021-01-05   TWLO  341.239990  341.239990  341.790009  329.000000   
4     2021-01-06   META  263.030914  263.309998  267.750000  260.010010   

Price        Open    Volume  
0      274.779999  15106100  
1      339.000000   1968900  
2      268.290009   9871600  
3      330.429993   1375400  
4      262.000000  24354100  


In [4]:
# We want to add some additional calculations to assist with analysis

# 50 and 200 Day moving average of the closing price

for cp in data_stacked:
    data_stacked['MA50'] = data_stacked.groupby('Ticker')['Close'].transform(lambda x: x.rolling(50).mean())
    data_stacked['MA200'] = data_stacked.groupby('Ticker')['Close'].transform(lambda x: x.rolling(200).mean())

# Previous day close

for cp in data_stacked:
    data_stacked['Previous day close'] = data_stacked.groupby('Ticker')['Close'].shift(1)

# Change in price
for cp in data_stacked:
    data_stacked['Change in price'] = data_stacked['Close'] - data_stacked['Previous day close']

# Percent change
for cp in data_stacked:
    data_stacked['Percent change price'] = data_stacked.groupby('Ticker')['Close'].pct_change()

# Previous day volume

for cp in data_stacked:
    data_stacked['Previous day volume'] = data_stacked.groupby('Ticker')['Volume'].shift(1)

# Change in volume

for cp in data_stacked:
    data_stacked['Change in volume'] = data_stacked['Volume'] - data_stacked['Previous day volume']

# Percentage change in volume
    
for cp in data_stacked:
    data_stacked['Percent change volume'] = data_stacked.groupby('Ticker')['Volume'].pct_change()

data_stacked.head()

Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,MA50,MA200,Previous day close,Change in price,Percent change price,Previous day volume,Change in volume,Percent change volume
0,2021-01-04,META,268.654968,268.940002,275.000000,265.200012,274.779999,15106100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-04,TWLO,334.589996,334.589996,340.596008,326.260010,339.000000,1968900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-05,META,270.682800,270.970001,272.399994,268.209991,268.290009,9871600,NaN,NaN,268.940002,2.029999,0.007548,15106100.0,-5234500.0,-0.346516
3,2021-01-05,TWLO,341.239990,341.239990,341.790009,329.000000,330.429993,1375400,NaN,NaN,334.589996,6.649994,0.019875,1968900.0,-593500.0,-0.301437
4,2021-01-06,META,263.030914,263.309998,267.750000,260.010010,262.000000,24354100,NaN,NaN,270.970001,-7.660004,-0.028269,9871600.0,14482500.0,1.467087


In [5]:
# Generate a CSV that we will use in a Tableau visualization

data_stacked.to_csv('stock_data.csv', index=False)

In [ ]:
# Create the figure object
fig, ax = plt.subplots(figsize=(14, 7))  # Set the figure size for better readability

# Plot the closing price for each ticker
for ticker in data_stacked['Ticker'].unique():
    df_ticker = data_stacked[data_stacked['Ticker'] == ticker]
    ax.plot(df_ticker['Date'], df_ticker['Close'], label=ticker)

# Setting the title and labels
ax.set_title('Closing Price Over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Closing Price (USD)')

# Adding a legend to distinguish between tickers
ax.legend()

# Improve readability of the x-axis dates
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()